In [74]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy
import joblib
import json

# AMBIL DATA DAN HAPUS KOLOM KOSONG

In [2]:
customer_interactions = 'customer_interactions.csv'

# Gunakan fungsi read_csv dari pandas untuk membaca file CSV ke dalam DataFrame
customer_interactions = pd.read_csv(customer_interactions)

# Loop melalui semua kolom di DataFrame
for nama_kolom in customer_interactions.columns:
    # Periksa apakah semua nilai di dalam kolom tersebut kosong
    if customer_interactions[nama_kolom].isnull().all():
        customer_interactions.drop(nama_kolom, axis=1, inplace=True)

# ----------------------------------------------------------------------------------------

product_details = 'product_details.csv'

# Gunakan fungsi read_csv dari pandas untuk membaca file CSV ke dalam DataFrame
product_details = pd.read_csv(product_details, sep=';')

# Loop melalui semua kolom di DataFrame
for nama_kolom in product_details.columns:
    # Periksa apakah semua nilai di dalam kolom tersebut kosong
    if product_details[nama_kolom].isnull().all():
        product_details.drop(nama_kolom, axis=1, inplace=True)
        
# ----------------------------------------------------------------------------------------

purchase_history = 'purchase_history.csv'

# Gunakan fungsi read_csv dari pandas untuk membaca file CSV ke dalam DataFrame
purchase_history = pd.read_csv(purchase_history, sep=';')

# Loop melalui semua kolom di DataFrame
for nama_kolom in purchase_history.columns:
    # Periksa apakah semua nilai di dalam kolom tersebut kosong
    if purchase_history[nama_kolom].isnull().all():
        purchase_history.drop(nama_kolom, axis=1, inplace=True)

In [3]:
customer_interactions

,customer_id,page_views,time_spent
0,1,25,120
1,2,20,90
2,3,30,150
3,4,15,80
4,5,22,110


In [4]:
product_details

,product_id,category,price,ratings
0,101,Electronics,500,4.5
1,102,Clothing,50,3.8
2,103,Home & Kitchen,200,4.2
3,104,Beauty,30,4.0
4,105,Electronics,800,4.8


In [5]:
purchase_history

,customer_id,product_id,purchase_date
0,1,101,2023-01-01
1,1,105,2023-01-05
2,2,102,2023-01-02
3,3,103,2023-01-03
4,4,104,2023-01-04
5,5,101,2023-01-05


# GENERATE DATA DUMMY

In [6]:
# Generate data dummy
np.random.seed(42)  # Untuk reproduksibilitas
customer_ids = np.random.randint(5, 11, size=10)
page_views = np.random.randint(10, 50, size=10)
time_spent = np.random.randint(5, 180, size=10)

# Buat DataFrame tambahan
df_customer_tambahan = pd.DataFrame({
    'customer_id': customer_ids,
    'page_views': page_views,
    'time_spent': time_spent
})

# Gabungkan dataframe menggunakan concat
customer_interactions = pd.concat([customer_interactions, df_customer_tambahan], ignore_index=True)
customer_interactions

,customer_id,page_views,time_spent
0,1,25,120
1,2,20,90
2,3,30,150
3,4,15,80
4,5,22,110
5,8,45,25
6,9,49,165
7,7,33,62
8,9,12,26
9,9,31,93


In [7]:
# Generate data dummy
np.random.seed(42)  # Untuk reproduksibilitas
product_ids = np.random.randint(105, 120, size=10)
categories = np.random.choice(['Electronics', 'Clothing', 'Home & Kitchen', 'Beauty'], size=10)
prices = np.random.randint(20, 1000, size=10)
ratings = np.random.uniform(3.0, 5.0, size=10)

# Buat DataFrame
df_product_details = pd.DataFrame({
    'product_id': product_ids,
    'category': categories,
    'price': prices,
    'ratings': ratings
})

# Gabungkan dataframe menggunakan concat
product_details = pd.concat([product_details, df_product_details], ignore_index=True)
product_details

,product_id,category,price,ratings
0,101,Electronics,500,4.500000
1,102,Clothing,50,3.800000
2,103,Home & Kitchen,200,4.200000
3,104,Beauty,30,4.000000
4,105,Electronics,800,4.800000
5,111,Home & Kitchen,681,3.366809
6,108,Home & Kitchen,328,3.608484
7,117,Home & Kitchen,789,4.049513
8,119,Home & Kitchen,363,3.863890
9,115,Beauty,511,3.582458


In [8]:
from datetime import datetime, timedelta

# Menghasilkan data dummy sebanyak 100 data
np.random.seed(42)  # Untuk kejelasan penghasilan data acak
customer_ids = np.random.randint(0, 11, size=500)
product_ids = np.random.randint(100, 120, size=500)
random_dates = [pd.Timestamp('2023-01-01') + pd.to_timedelta(np.random.randint(0, 30), 'D') for _ in range(500)]


dummy_data = pd.DataFrame({
    'customer_id': customer_ids,
    'product_id': product_ids,
    'purchase_date': random_dates
})

# Ambil tanggal saja tanpa jam
dummy_data['purchase_date'] = dummy_data['purchase_date'].dt.date

# Gabungkan dataframe menggunakan concat
purchase_history = pd.concat([purchase_history, dummy_data], ignore_index=True)
purchase_history

,customer_id,product_id,purchase_date
0,1,101,2023-01-01
1,1,105,2023-01-05
2,2,102,2023-01-02
3,3,103,2023-01-03
4,4,104,2023-01-04
...,...,...,...
501,9,107,2023-01-24
502,7,104,2023-01-17
503,9,114,2023-01-04
504,9,103,2023-01-29


# PENANGANAN DUPLIKAT

In [9]:
# Menghitung jumlah duplikat berdasarkan kolom 'customer_id'
jumlah_duplikat_sebelum = customer_interactions.duplicated(subset='customer_id').sum()

# Menampilkan jumlah duplikat sebelum dihapus
print("Jumlah duplikat Customers:", jumlah_duplikat_sebelum)

# Pengecekan dan penghapusan duplikat berdasarkan kolom 'customer_id'
customer_interactions.drop_duplicates(subset='customer_id', keep='first', inplace=True)

Jumlah duplikat Customers: 6


In [10]:
# Menghitung jumlah duplikat berdasarkan kolom 'customer_id'
jumlah_duplikat_sebelum = product_details.duplicated(subset='product_id').sum()

# Menampilkan jumlah duplikat sebelum dihapus
print("Jumlah duplikat Products:", jumlah_duplikat_sebelum)

# Pengecekan dan penghapusan duplikat berdasarkan kolom 'customer_id'
product_details.drop_duplicates(subset='product_id', keep='first', inplace=True)

Jumlah duplikat Products: 2


# MERGE DATA UNTUK DIJADIKAN DATA SET UTAMA YANG LEBIH BANYAK

In [11]:
# # Gabungkan data dari tiga tabel
merged_data = pd.merge(purchase_history, customer_interactions, on='customer_id')
merged_data = pd.merge(merged_data, product_details, on='product_id')

# # Melakukan pembulatan untuk kolom 'ratings' menjadi satu angka setelah koma
# merged_data['ratings'] = merged_data['ratings'].round(1)
merged_data

,customer_id,product_id,purchase_date,page_views,time_spent,category,price,ratings
0,1,101,2023-01-01,25,120,Electronics,500,4.500000
1,2,101,2023-01-07,20,90,Electronics,500,4.500000
2,2,101,2023-01-16,20,90,Electronics,500,4.500000
3,2,101,2023-01-18,20,90,Electronics,500,4.500000
4,2,101,2023-01-19,20,90,Electronics,500,4.500000
...,...,...,...,...,...,...,...,...
271,7,109,2023-01-25,33,62,Beauty,405,3.584289
272,7,109,2023-01-05,33,62,Beauty,405,3.584289
273,8,109,2023-01-17,45,25,Beauty,405,3.584289
274,8,109,2023-01-11,45,25,Beauty,405,3.584289


In [12]:
# Menyimpan DataFrame ke dalam file CSV
merged_data.to_csv('merged_data.csv', index=False)

print("Data berhasil disimpan ke dalam file CSV.")

Data berhasil disimpan ke dalam file CSV.


In [13]:
# Check for null values in the entire DataFrame
null_values = merged_data.isnull().sum()

# Display the total count of null values in the DataFrame
total_null_count = null_values
print("Total Count of Null Values:\n",total_null_count)

Total Count of Null Values:
 customer_id      0
product_id       0
purchase_date    0
page_views       0
time_spent       0
category         0
price            0
ratings          0
dtype: int64


# MODEL

In [14]:
# Membaca dataset dari DataFrame
data = merged_data.copy()
reader = Reader(rating_scale=(1, 5))

# Membuat objek Dataset dari DataFrame
dataset = Dataset.load_from_df(data[['customer_id', 'product_id', 'ratings']], reader)

# Membagi data menjadi set pelatihan dan set validasi
trainset, testset = train_test_split(dataset, test_size=0.2, random_state=42)

# Membangun model SVD (Singular Value Decomposition)
model = SVD()
model.fit(trainset)

# Menguji model pada set validasi
predictions = model.test(testset)

# Mengukur akurasi model
accuracy.rmse(predictions)

# Simpan model ke dalam file h5
model_filename = 'svd_model_predictive_analytics.h5'
joblib.dump(model, model_filename)

print(f"Model SVD telah disimpan dalam file: {model_filename}")

RMSE: 0.1125
Model SVD telah disimpan dalam file: svd_model_predictive_analytics.h5


# PREDIKSI

In [45]:
def get_top_n_recommendations(data, model, customer_id, n=5):
    # Memeriksa apakah customer_id valid
    if customer_id not in data['customer_id'].unique():
        print(f"Error: Customer ID {customer_id} tidak valid.")
        return

    # Mengumpulkan produk yang belum dilihat oleh pelanggan
    products_not_seen = data.loc[~data['product_id'].isin(data[data['customer_id'] == customer_id]['product_id'])]['product_id'].unique()

    # Memeriksa apakah ada produk yang belum dilihat
    if not products_not_seen.any():
        print(f"Error: Pelanggan {customer_id} sudah melihat semua produk.")
        return

    # Membuat prediksi untuk produk yang belum dilihat
    pred_list = [(prod_id, model.predict(customer_id, prod_id).est) for prod_id in products_not_seen]

    # Normalisasi skor prediksi antara 0 dan 1
    min_pred, max_pred = min(pred_list, key=lambda x: x[1])[1], max(pred_list, key=lambda x: x[1])[1]

    # Memeriksa apakah semua nilai prediksi sama (menghindari pembagian dengan nol)
    if min_pred == max_pred:
        # Semua nilai prediksi sama, memberikan skor tetap
        normalized_preds = [(prod_id, 0.5) for prod_id, _ in pred_list]
    else:
        normalized_preds = [(prod_id, (pred - min_pred) / (max_pred - min_pred)) for prod_id, pred in pred_list]

    # Mengurutkan produk berdasarkan skor prediksi
    sorted_products = sorted(normalized_preds, key=lambda x: x[1], reverse=True)

    # Mengambil N produk teratas
    top_n = [prod_id for prod_id, _ in sorted_products[:n]]

    # Mengembalikan ID produk yang direkomendasikan beserta skor prediksi
    return sorted_products[:n]

In [54]:
import joblib

# Muat kembali model dari file h5
loaded_model = joblib.load('svd_model_predictive_analytics.h5')

# Contoh penggunaan
customer_id_to_recommend = 1
recommendations = get_top_n_recommendations(data, loaded_model, customer_id_to_recommend)

# Memeriksa apakah hasil rekomendasi valid
if recommendations:
    print(f"Rekomendasi produk untuk pelanggan {customer_id_to_recommend}: {recommendations}")

Rekomendasi produk untuk pelanggan 1: [(109, 0.5)]
